In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [10]:
import torch
from fastai.conv_learner import *
from fastai.dataset import *
from fastai.models.resnet import vgg_resnet50

In [11]:
x = torch.ones([2, 3, 128, 128])

In [12]:
x = torch.randn([2, 3, 128, 128])

In [13]:
y = torch.ones([2,128,128])

In [15]:
yv = VV(y.long())

In [16]:
xv = VV(x)

In [17]:
F.nll_loss(xv, yv)

Variable containing:
1.00000e-03 *
  5.3711
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [19]:
xv.shape

torch.Size([2, 3, 128, 128])

In [21]:
type(yv.data)

torch.cuda.LongTensor

In [23]:
type(xv.data)

torch.cuda.FloatTensor

### WTF

In [29]:
f = resnet34
cut,lr_cut = model_meta[f]

In [27]:
def get_base():
    layers = cut_model(f(True), cut)
    return nn.Sequential(*layers)

In [24]:
class StdUpsample(nn.Module):
    def __init__(self, nin, nout):
        super().__init__()
        self.conv = nn.ConvTranspose2d(nin, nout, 2, stride=2)
        self.bn = nn.BatchNorm2d(nout)
        
    def forward(self, x): return self.bn(F.relu(self.conv(x)))

In [25]:
class Upsample34(nn.Module):
    def __init__(self, rn):
        super().__init__()
        self.rn = rn
        self.features = nn.Sequential(
            rn, nn.ReLU(),
            StdUpsample(512,256),
            StdUpsample(256,256),
            StdUpsample(256,256),
            StdUpsample(256,256),
#             nn.ConvTranspose2d(256, 1, 2, stride=2))
            nn.ConvTranspose2d(256, 3, 2, stride=2))
        
    def forward(self,x): 
        print('Forward pass called')
        out = self.features(x)
        print(out.shape)
#         xreshape = x.view(x.shape[0], x.shape[1], -1)
#         print(xreshape.shape)
        return F.log_softmax(out)

In [31]:
m_base = get_base()
m = to_gpu(Upsample34(m_base))

In [32]:
out = m(xv)

Forward pass called
torch.Size([2, 3, 128, 128])


In [33]:
F.nll_loss(out, yv)

Variable containing:
 1.1278
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [35]:
x.shape

torch.Size([2, 3, 128, 128])

In [39]:
a,b = torch.max(x, 1)

In [40]:
b.shape

torch.Size([2, 128, 128])

In [41]:
b


( 0 ,.,.) = 
   0   2   2  ...    2   0   1
   2   0   2  ...    2   0   0
   1   2   2  ...    1   0   2
     ...       ⋱       ...    
   1   1   1  ...    1   2   0
   2   0   2  ...    2   1   2
   0   2   1  ...    2   0   0

( 1 ,.,.) = 
   0   1   2  ...    2   2   1
   0   0   0  ...    1   1   2
   1   1   0  ...    0   0   0
     ...       ⋱       ...    
   0   2   1  ...    2   1   2
   0   2   1  ...    1   2   2
   1   1   1  ...    1   0   2
[torch.LongTensor of size 2x128x128]